In [251]:
# Establish database connections
import datetime
import time
import requests
import string
import io
import pprint as pp
import json
import pandas as pd
import sqlalchemy
from sqlalchemy import text
import psycopg2
from datetime import datetime
import config_warehouse_RDS as creds_warehouse
import config_lake_RDS_locations as creds_lake

# Establish database connections

In [245]:
DB_LOGIN_lake = {
    "db_host": creds_lake.DB_HOST,
    "db_name": creds_lake.DB_NAME,
    "db_port": creds_lake.DB_PORT,
    "db_user": creds_lake.DB_USER,
    "db_pw": creds_lake.DB_PW}

DB_TABLE_lake = creds_lake.DB_TABLE

In [252]:
DB_LOGIN_warehouse = {
    "db_host": creds_warehouse.DB_HOST,
    "db_name": creds_warehouse.DB_NAME,
    "db_port": creds_warehouse.DB_PORT,
    "db_user": creds_warehouse.DB_USER,
    "db_pw": creds_warehouse.DB_PW}

DB_TABLE_warehouse = creds_warehouse.DB_TABLE_4

In [153]:
# function to establish db connection (engine)
def create_db_engine(db_host, db_port, db_name, db_user, db_pw):
        engine_url = f"postgresql://{db_user}:{db_pw}@{db_host}:{db_port}/{db_name}"
        engine = sqlalchemy.create_engine(engine_url)
        return engine

In [53]:
# Connect to DB engine lake
engine_lake = create_db_engine(**DB_LOGIN_lake)
engine_lake.connect()

In [357]:
# Connect to DB engine warehouse
engine_warehouse = create_db_engine(**DB_LOGIN_warehouse)
engine_warehouse.connect()

In [ ]:
query_all = """
    SELECT *
    FROM chargingstaions
"""

df = pd.read_sql(query_all,con=engine_lake)

In [2]:
#df = pd.read_csv("chargingstations_202212091604.csv")

In [3]:
df.head(3)

,EVSEStatusRecord,OperatorID,OperatorName,EvseID,EVSEStatus,0,datetime
0,NaN,CH*CCC,Move,CH*CCI*E20117,Available,NaN,20221104-174216
1,NaN,CH*CCC,Move,CH*CCC*E50320,Available,NaN,20221104-174216
2,NaN,CH*CCC,Move,CH*CCC*E50319,Available,NaN,20221104-174216


In [6]:
df_red = df[["EvseID","EVSEStatus","datetime"]]

In [7]:
df_red.head(3)

,EvseID,EVSEStatus,datetime
0,CH*CCI*E20117,Available,20221104-174216
1,CH*CCC*E50320,Available,20221104-174216
2,CH*CCC*E50319,Available,20221104-174216
3,CH*CCI*E20115,Available,20221104-174216
4,CH*CCI*E20118,Available,20221104-174216
...,...,...,...
377710288,CH*CCI*E20120,Available,20221104-174216
377710289,CH*CCI*E20123,Available,20221104-174216
377710290,CH*CCI*E20125,Available,20221104-174216
377710291,CH*CCI*E20124,Available,20221104-174216


In [8]:
df_sorted = df_red.sort_values(by=["EvseID","datetime"])

In [9]:
df_sorted.head(-5)

,EvseID,EVSEStatus,datetime
306963349,+41*001*10*1,OutOfService,20221030-080929
306972734,+41*001*10*1,OutOfService,20221030-081957
306982118,+41*001*10*1,OutOfService,20221030-083836
306991503,+41*001*10*1,Unknown,20221030-095610
307000888,+41*001*10*1,Unknown,20221030-095814
...,...,...,...
306937236,NaN,NaN,20221127-110516
306939579,NaN,NaN,20221127-110516
306939662,NaN,NaN,20221127-110516
306941302,NaN,NaN,20221127-110516


In [10]:
df_sorted.to_csv("sorted_status.csv")

In [11]:
df_sorted.dropna(inplace=True)

In [12]:
df_sorted.head(-5)

,EvseID,EVSEStatus,datetime
306963349,+41*001*10*1,OutOfService,20221030-080929
306972734,+41*001*10*1,OutOfService,20221030-081957
306982118,+41*001*10*1,OutOfService,20221030-083836
306991503,+41*001*10*1,Unknown,20221030-095610
307000888,+41*001*10*1,Unknown,20221030-095814
...,...,...,...
306868587,CHFASE4130204,Available,20221127-105716
306878351,CHFASE4130204,Available,20221127-105816
306888124,CHFASE4130204,Available,20221127-105915
306897897,CHFASE4130204,Available,20221127-110016


In [13]:
df_sorted["EVSEStatus_prev"] = df_sorted["EVSEStatus"]
df_sorted['EVSEStatus_prev'] = df_sorted['EVSEStatus_prev'].shift(periods=1,fill_value=0)

In [14]:
df_sorted_final = df_sorted.loc[(df_sorted["EVSEStatus"] == "Occupied") & (df_sorted["EVSEStatus_prev"] != "Occupied")]

In [28]:
df_sorted_occupied = df_sorted.loc[(df_sorted["EVSEStatus"] == "Occupied")]

In [15]:
df_sorted_final['datetime'] = pd.to_datetime(df_sorted_final['datetime'])
df_sorted_final['datetime'] = df_sorted_final['datetime'].dt.date

/var/folders/b5/yxz7l5d156v4d4bptl2rk_f40000gn/T/ipykernel_91175/1807601244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sorted_final['datetime'] = pd.to_datetime(df_sorted_final['datetime'])
/var/folders/b5/yxz7l5d156v4d4bptl2rk_f40000gn/T/ipykernel_91175/1807601244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sorted_final['datetime'] = df_sorted_final['datetime'].dt.date


In [29]:
df_sorted_occupied['datetime'] = pd.to_datetime(df_sorted_occupied['datetime'])
df_sorted_occupied['datetime'] = df_sorted_occupied['datetime'].dt.date

/var/folders/b5/yxz7l5d156v4d4bptl2rk_f40000gn/T/ipykernel_91175/328788843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sorted_occupied['datetime'] = pd.to_datetime(df_sorted_occupied['datetime'])
/var/folders/b5/yxz7l5d156v4d4bptl2rk_f40000gn/T/ipykernel_91175/328788843.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sorted_occupied['datetime'] = df_sorted_occupied['datetime'].dt.date


In [16]:
df_number_of_cars_day = df_sorted_final.groupby(['EvseID','datetime']).count().reset_index()

In [17]:
df_number_of_cars_day = df_number_of_cars_day.rename(columns={"EVSEStatus":"day_cars"})

In [19]:
df_number_of_cars_day = df_number_of_cars_day[["EvseID","day_cars","datetime"]]

In [20]:
df_number_of_cars_day

,EvseID,day_cars,datetime
0,+41*001*10*1,3,2022-10-30
1,+41*001*10*1,10,2022-11-03
2,+41*001*10*1,2,2022-11-07
3,+41*001*10*1,2,2022-11-09
4,+41*001*10*1,1,2022-11-11
...,...,...,...
95795,CHFASE4130204,6,2022-11-23
95796,CHFASE4130204,1,2022-11-24
95797,CHFASE4130204,4,2022-11-25
95798,CHFASE4130204,3,2022-11-26


In [22]:
df_number_of_cars_per_month = df_number_of_cars_day.groupby(['EvseID']).sum().reset_index()

/var/folders/b5/yxz7l5d156v4d4bptl2rk_f40000gn/T/ipykernel_91175/80092205.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_number_of_cars_per_month = df_number_of_cars_day.groupby(['EvseID']).sum().reset_index()


In [23]:
df_number_of_cars_per_month

,EvseID,day_cars
0,+41*001*10*1,18
1,+41*001*41*1,3
2,+41*001*75*1,68
3,+41*001*76*1,76
4,+41*001*77*1,18
...,...,...
7606,CHFASE4130104,100
7607,CHFASE4130201,40
7608,CHFASE4130202,52
7609,CHFASE4130203,93


In [24]:
df_number_of_cars_per_month["cars_per_day"] = df_number_of_cars_per_month["day_cars"] / 28

In [25]:
df_number_of_cars_per_month

,EvseID,day_cars,cars_per_day
0,+41*001*10*1,18,0.642857
1,+41*001*41*1,3,0.107143
2,+41*001*75*1,68,2.428571
3,+41*001*76*1,76,2.714286
4,+41*001*77*1,18,0.642857
...,...,...,...
7606,CHFASE4130104,100,3.571429
7607,CHFASE4130201,40,1.428571
7608,CHFASE4130202,52,1.857143
7609,CHFASE4130203,93,3.321429


In [26]:
df_number_of_cars_day.to_csv("number_of_cars_day.csv")

In [27]:
df_number_of_cars_per_month.to_csv("number_of_cars_per_day.csv")

In [30]:
df_sorted_occupied.head(-3)

,EvseID,EVSEStatus,datetime,EVSEStatus_prev
310435723,+41*001*10*1,Occupied,2022-10-30,Available
310445108,+41*001*10*1,Occupied,2022-10-30,Occupied
310454492,+41*001*10*1,Occupied,2022-10-30,Occupied
310463877,+41*001*10*1,Occupied,2022-10-30,Occupied
310473262,+41*001*10*1,Occupied,2022-10-30,Occupied
...,...,...,...,...
302063040,CHFASE4130204,Occupied,2022-11-27,Occupied
302072807,CHFASE4130204,Occupied,2022-11-27,Occupied
302082580,CHFASE4130204,Occupied,2022-11-27,Occupied
302092354,CHFASE4130204,Occupied,2022-11-27,Occupied


In [32]:
df_sorted_occupied = df_sorted_occupied[["EvseID","datetime","EVSEStatus"]]

In [33]:
df_sorted_occupied.head(-3)

,EvseID,datetime,EVSEStatus
310435723,+41*001*10*1,2022-10-30,Occupied
310445108,+41*001*10*1,2022-10-30,Occupied
310454492,+41*001*10*1,2022-10-30,Occupied
310463877,+41*001*10*1,2022-10-30,Occupied
310473262,+41*001*10*1,2022-10-30,Occupied
...,...,...,...
302063040,CHFASE4130204,2022-11-27,Occupied
302072807,CHFASE4130204,2022-11-27,Occupied
302082580,CHFASE4130204,2022-11-27,Occupied
302092354,CHFASE4130204,2022-11-27,Occupied


In [36]:
df_occupied_days = df_sorted_occupied.groupby(["EvseID","datetime"]).count().reset_index()

In [38]:
df_occupied_days.head(-3)

,EvseID,datetime,EVSEStatus
0,+41*001*10*1,2022-10-30,25
1,+41*001*10*1,2022-11-03,249
2,+41*001*10*1,2022-11-07,157
3,+41*001*10*1,2022-11-09,201
4,+41*001*10*1,2022-11-11,2
...,...,...,...
99200,CHFASE4130204,2022-11-20,34
99201,CHFASE4130204,2022-11-21,13
99202,CHFASE4130204,2022-11-22,38
99203,CHFASE4130204,2022-11-23,96


In [39]:
df_occupied_days.to_csv("occupied_day.csv")

In [40]:
df_occupied_per_day = df_occupied_days.groupby(['EvseID']).sum().reset_index()

/var/folders/b5/yxz7l5d156v4d4bptl2rk_f40000gn/T/ipykernel_91175/3175514191.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_occupied_per_day = df_occupied_days.groupby(['EvseID']).sum().reset_index()


In [41]:
df_occupied_per_day.head(-3)

,EvseID,EVSEStatus
0,+41*001*10*1,634
1,+41*001*41*1,148
2,+41*001*75*1,2254
3,+41*001*76*1,3161
4,+41*001*77*1,839
...,...,...
7605,CHFASE4130101,11795
7606,CHFASE4130102,11723
7607,CHFASE4130103,3111
7608,CHFASE4130104,1891


In [65]:
df_occupied_per_day["daily_occupancy"] = df_occupied_per_day["EVSEStatus"] / 28

In [66]:
df_occupied_per_day.head(-3)

,EvseID,EVSEStatus,daily_cars,daily_occupancy
0,+41*001*10*1,634,22.642857,22.642857
1,+41*001*41*1,148,5.285714,5.285714
2,+41*001*75*1,2254,80.500000,80.500000
3,+41*001*76*1,3161,112.892857,112.892857
4,+41*001*77*1,839,29.964286,29.964286
...,...,...,...,...
7605,CHFASE4130101,11795,421.250000,421.250000
7606,CHFASE4130102,11723,418.678571,418.678571
7607,CHFASE4130103,3111,111.107143,111.107143
7608,CHFASE4130104,1891,67.535714,67.535714


In [46]:
df_occupied_per_day.describe()

,EVSEStatus,daily_cars
count,7613.000000,7613.000000
mean,3972.162485,141.862946
std,5310.132975,189.647606
min,1.000000,0.035714
25%,691.000000,24.678571
50%,2239.000000,79.964286
75%,5180.000000,185.000000
max,40354.000000,1441.214286


In [47]:
1441/60

24.016666666666666

In [67]:
df_occupied_per_day.to_csv("occupied_daily.csv")

In [57]:
df_merged_day = pd.merge(left=df_number_of_cars_day,left_on="EvseID",right=df_occupied_days,right_on="EvseID")

In [58]:
df_merged_day

,EvseID,day_cars,datetime_x,datetime_y,EVSEStatus
0,+41*001*10*1,3,2022-10-30,2022-10-30,25
1,+41*001*10*1,3,2022-10-30,2022-11-03,249
2,+41*001*10*1,3,2022-10-30,2022-11-07,157
3,+41*001*10*1,3,2022-10-30,2022-11-09,201
4,+41*001*10*1,3,2022-10-30,2022-11-11,2
...,...,...,...,...,...
1823086,CHFASE4130204,1,2022-11-27,2022-11-23,96
1823087,CHFASE4130204,1,2022-11-27,2022-11-24,30
1823088,CHFASE4130204,1,2022-11-27,2022-11-25,96
1823089,CHFASE4130204,1,2022-11-27,2022-11-26,64


In [61]:
df_merged_day = df_merged_day[["EvseID","datetime_x","day_cars","EVSEStatus"]]

In [62]:
df_merged_day.rename(columns={"EvseID":"evse_id","datetime_x":"date","EVSEStatus":"day_occupancy"},inplace=True)

/var/folders/b5/yxz7l5d156v4d4bptl2rk_f40000gn/T/ipykernel_91175/2816314137.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_day.rename(columns={"EvseID":"evse_id","datetime_x":"date","EVSEStatus":"day_occupancy"},inplace=True)


In [64]:
df_merged_day.head(-3)

,evse_id,date,day_cars,day_occupancy
0,+41*001*10*1,2022-10-30,3,25
1,+41*001*10*1,2022-10-30,3,249
2,+41*001*10*1,2022-10-30,3,157
3,+41*001*10*1,2022-10-30,3,201
4,+41*001*10*1,2022-10-30,3,2
...,...,...,...,...
1823083,CHFASE4130204,2022-11-27,1,34
1823084,CHFASE4130204,2022-11-27,1,13
1823085,CHFASE4130204,2022-11-27,1,38
1823086,CHFASE4130204,2022-11-27,1,96


In [68]:
df_merged_days = pd.merge(left=df_number_of_cars_per_month,left_on="EvseID",right=df_occupied_per_day,right_on="EvseID")

In [69]:
df_merged_days

,EvseID,day_cars,cars_per_day,EVSEStatus,daily_cars,daily_occupancy
0,+41*001*10*1,18,0.642857,634,22.642857,22.642857
1,+41*001*41*1,3,0.107143,148,5.285714,5.285714
2,+41*001*75*1,68,2.428571,2254,80.500000,80.500000
3,+41*001*76*1,76,2.714286,3161,112.892857,112.892857
4,+41*001*77*1,18,0.642857,839,29.964286,29.964286
...,...,...,...,...,...,...
7606,CHFASE4130104,100,3.571429,1891,67.535714,67.535714
7607,CHFASE4130201,40,1.428571,961,34.321429,34.321429
7608,CHFASE4130202,52,1.857143,861,30.750000,30.750000
7609,CHFASE4130203,93,3.321429,2105,75.178571,75.178571


In [70]:
df_merged_days = df_merged_days[["EvseID","cars_per_day","daily_occupancy"]]

In [71]:
df_merged_days.rename(columns={"EvseID":"evse_id","cars_per_day":"daily_cars"},inplace=True)

/var/folders/b5/yxz7l5d156v4d4bptl2rk_f40000gn/T/ipykernel_91175/2851057114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_days.rename(columns={"EvseID":"evse_id","cars_per_day":"daily_cars"},inplace=True)


In [72]:
df_merged_days

,evse_id,daily_cars,daily_occupancy
0,+41*001*10*1,0.642857,22.642857
1,+41*001*41*1,0.107143,5.285714
2,+41*001*75*1,2.428571,80.500000
3,+41*001*76*1,2.714286,112.892857
4,+41*001*77*1,0.642857,29.964286
...,...,...,...
7606,CHFASE4130104,3.571429,67.535714
7607,CHFASE4130201,1.428571,34.321429
7608,CHFASE4130202,1.857143,30.750000
7609,CHFASE4130203,3.321429,75.178571


# Load static data from lake

In [54]:
query_all = """
    SELECT *
    FROM chargingstations_static
"""

df_static = pd.read_sql(query_all,con=engine_lake)

In [55]:
df_static_renamed = df_static.rename(columns={"OperatorID":"operator_id","EvseID":"evse_id","ChargingStationId":"charging_station_id","City":"city","Street":"street"})

In [160]:
df_static_renamed.head(-3)

,operator_id,OperatorName,charging_station_id,evse_id,city,PostalCode,street,power
0,CH*EWD,EWD Elektrizitätswerk Davos AG,CH*EWD*E123*0033,CH*EWD*E123*0033,Davos,7260,Bahnhofstrasse,22.0
1,CH*EWD,EWD Elektrizitätswerk Davos AG,CH*EWD*E123*0027,CH*EWD*E123*0027,Davos,7260,Bahnhofstrasse,22.0
2,CH*EWD,EWD Elektrizitätswerk Davos AG,CH*EWD*E123*0023,CH*EWD*E123*0023,Davos,7270,Talstrasse,22.0
3,CH*EWD,EWD Elektrizitätswerk Davos AG,CH*EWD*E123*0032,CH*EWD*E123*0032,Davos,7270,Talstrasse,22.0
4,CH*EWD,EWD Elektrizitätswerk Davos AG,CH*EWD*E123*0005,CH*EWD*E123*0005,Davos,7270,Talstrasse,22.0
...,...,...,...,...,...,...,...,...
7403,CH*FASTNED,Fastned,50131,CHFASE4130101,Suhr,5034,Rastplatz Suhr A1,300.0
7404,CH*FASTNED,Fastned,50131,CHFASE4130104,Suhr,5034,Rastplatz Suhr A1,50.0
7405,CH*FASTNED,Fastned,50231,CHFASE4130203,Lenzburg,5600,Rastplatz Lenzburg A1,300.0
7406,CH*FASTNED,Fastned,50231,CHFASE4130202,Lenzburg,5600,Rastplatz Lenzburg A1,50.0


In [73]:
df_merged_day_complete = pd.merge(left=df_merged_day,left_on="evse_id",right=df_static_renamed[["evse_id","power"]],right_on="evse_id")

In [74]:
df_merged_day_complete.head()

,evse_id,date,day_cars,day_occupancy,power
0,+41*001*10*1,2022-10-30,3,25,11.0
1,+41*001*10*1,2022-10-30,3,249,11.0
2,+41*001*10*1,2022-10-30,3,157,11.0
3,+41*001*10*1,2022-10-30,3,201,11.0
4,+41*001*10*1,2022-10-30,3,2,11.0


In [75]:
df_merged_day_complete["day_kWh"] = (df_merged_day_complete["day_occupancy"] / 60 )* (df_merged_day_complete["power"])

In [76]:
df_merged_days_complete = pd.merge(left=df_merged_days,left_on="evse_id",right=df_static_renamed[["evse_id","power"]],right_on="evse_id")

In [77]:
df_merged_days_complete.head()

,evse_id,daily_cars,daily_occupancy,power
0,+41*001*10*1,0.642857,22.642857,11.0
1,+41*001*41*1,0.107143,5.285714,11.0
2,+41*001*75*1,2.428571,80.500000,11.0
3,+41*001*76*1,2.714286,112.892857,11.0
4,+41*001*77*1,0.642857,29.964286,11.0


In [78]:
df_merged_days_complete["daily_kWh"] = (df_merged_days_complete["daily_occupancy"] / 60 )* (df_merged_days_complete["power"])

In [79]:
df_merged_days_complete

,evse_id,daily_cars,daily_occupancy,power,daily_kWh
0,+41*001*10*1,0.642857,22.642857,11.0,4.151190
1,+41*001*41*1,0.107143,5.285714,11.0,0.969048
2,+41*001*75*1,2.428571,80.500000,11.0,14.758333
3,+41*001*76*1,2.714286,112.892857,11.0,20.697024
4,+41*001*77*1,0.642857,29.964286,11.0,5.493452
...,...,...,...,...,...
5185,CHFASE4130104,3.571429,67.535714,50.0,56.279762
5186,CHFASE4130201,1.428571,34.321429,300.0,171.607143
5187,CHFASE4130202,1.857143,30.750000,50.0,25.625000
5188,CHFASE4130203,3.321429,75.178571,300.0,375.892857


In [80]:
df_merged_days_complete.to_csv("df_merged_days_complete.csv")

In [81]:
df_merged_day_complete.to_csv("df_merged_day_complete.csv")

# Load data into warehouse

In [105]:
df_merged_days_complete = pd.read_csv("df_merged_days_complete.csv")

In [106]:
df_merged_days_complete = df_merged_days_complete[["evse_id","daily_cars","daily_occupancy","daily_kWh"]]

In [102]:
df_merged_days_complete.head()

,evse_id,daily_cars,daily_occupancy,daily_kWh
0,+41*001*10*1,0.642857,22.642857,4.151190
1,+41*001*41*1,0.107143,5.285714,0.969048
2,+41*001*75*1,2.428571,80.500000,14.758333
3,+41*001*76*1,2.714286,112.892857,20.697024
4,+41*001*77*1,0.642857,29.964286,5.493452


In [107]:
df_merged_days_complete.rename(columns={"daily_kWh":"daily_kwh"},inplace=True)

In [108]:
df_merged_days_complete

,evse_id,daily_cars,daily_occupancy,daily_kwh
0,+41*001*10*1,0.642857,22.642857,4.151190
1,+41*001*41*1,0.107143,5.285714,0.969048
2,+41*001*75*1,2.428571,80.500000,14.758333
3,+41*001*76*1,2.714286,112.892857,20.697024
4,+41*001*77*1,0.642857,29.964286,5.493452
...,...,...,...,...
5185,CHFASE4130104,3.571429,67.535714,56.279762
5186,CHFASE4130201,1.428571,34.321429,171.607143
5187,CHFASE4130202,1.857143,30.750000,25.625000
5188,CHFASE4130203,3.321429,75.178571,375.892857


In [110]:
df_merged_days_complete.describe()

,daily_cars,daily_occupancy,daily_kwh
count,5190.000000,5190.000000,5176.000000
mean,1.805884,132.459441,62.579361
std,6.694788,197.460874,111.452006
min,0.035714,0.035714,0.000000
25%,0.214286,20.401786,7.186012
50%,0.714286,69.428571,25.532440
75%,1.750000,163.366071,67.623810
max,193.678571,1441.214286,2106.250000


In [141]:
df_merged_days_complete = df_merged_days_complete.dropna()

In [115]:
df_merged_days_complete.describe()

,daily_cars,daily_occupancy,daily_kwh
count,5115.000000,5115.000000,5101.000000
mean,1.806082,132.806514,63.499465
std,6.739126,198.278692,112.007954
min,0.035714,0.035714,0.010714
25%,0.214286,20.482143,7.714286
50%,0.714286,69.535714,26.369048
75%,1.750000,163.303571,68.776190
max,193.678571,1441.214286,2106.250000


In [127]:
df_merged_days_complete = df_merged_days_complete.dropna(how='all', axis=1)

In [131]:
df_merged_days_complete

,evse_id,daily_cars,daily_occupancy,daily_kwh
0,+41*001*10*1,0.642857,22.642857,4.151190
1,+41*001*41*1,0.107143,5.285714,0.969048
2,+41*001*75*1,2.428571,80.500000,14.758333
3,+41*001*76*1,2.714286,112.892857,20.697024
4,+41*001*77*1,0.642857,29.964286,5.493452
...,...,...,...,...
5185,CHFASE4130104,3.571429,67.535714,56.279762
5186,CHFASE4130201,1.428571,34.321429,171.607143
5187,CHFASE4130202,1.857143,30.750000,25.625000
5188,CHFASE4130203,3.321429,75.178571,375.892857


In [145]:
df_merged_days_complete.to_sql(name=DB_TABLE_warehouse, con=engine_warehouse, if_exists="replace", index=False)

101

In [292]:
df_merged_day_complete

,evse_id,date,day_cars,day_occupancy,power,day_kWh
0,+41*001*10*1,2022-10-30,3,25,11.0,4.583333
1,+41*001*10*1,2022-10-30,3,249,11.0,45.650000
2,+41*001*10*1,2022-10-30,3,157,11.0,28.783333
3,+41*001*10*1,2022-10-30,3,201,11.0,36.850000
4,+41*001*10*1,2022-10-30,3,2,11.0,0.366667
...,...,...,...,...,...,...
1133450,CHFASE4130204,2022-11-27,1,96,50.0,80.000000
1133451,CHFASE4130204,2022-11-27,1,30,50.0,25.000000
1133452,CHFASE4130204,2022-11-27,1,96,50.0,80.000000
1133453,CHFASE4130204,2022-11-27,1,64,50.0,53.333333


In [298]:
df_merged_day_complete.rename(columns={"day_kWh":"day_kwh"},inplace=True)

In [301]:
df_merged_day_complete

,evse_id,date,day_cars,day_occupancy,day_kwh
0,+41*001*10*1,2022-10-30,3,25,4.583333
1,+41*001*10*1,2022-10-30,3,249,45.650000
2,+41*001*10*1,2022-10-30,3,157,28.783333
3,+41*001*10*1,2022-10-30,3,201,36.850000
4,+41*001*10*1,2022-10-30,3,2,0.366667
...,...,...,...,...,...
1133450,CHFASE4130204,2022-11-27,1,96,80.000000
1133451,CHFASE4130204,2022-11-27,1,30,25.000000
1133452,CHFASE4130204,2022-11-27,1,96,80.000000
1133453,CHFASE4130204,2022-11-27,1,64,53.333333


In [300]:
df_merged_day_complete = df_merged_day_complete[["evse_id","date","day_cars","day_occupancy","day_kwh"]]

In [361]:
df_merged_day_complete =df_merged_day_complete[df_merged_day_complete['evse_id'] != "+41*002*152*1"]

In [1]:
df_merged_day_complete.to_sql(name="charging_stations_occupancy_day", con=engine_warehouse, if_exists="append", index=False)